In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import pandas as pd
import os
import sys
import cv2
import numpy as np
sys.path.append('..')
import matplotlib.pyplot as plt
from fidelity_check import fidelity_detect, check_utils

### Check the assumption of archive's writes are always a subset of live

### Fidelity check

In [33]:
# dirr = '../revert_rewrite/writes/www.paulkasmingallery.com_641_3_4c30cf2b47f220b75ebeb15b727ca9b0594ed2a2a1a778d3e700940540db2017'
dirr = '../revert_rewrite/test/load_override/writes/golfdigest.com'
results = json.load(open(os.path.join(dirr, 'results.json'), 'r'))
stage, fixedIdx = None, None
for stage, result in results.items():
    if result['fixedIdx'] == -1:
        continue
    fixedIdx = result['fixedIdx']
    break
# stage = 'interaction_2'
# result = results[stage]
# fixedIdx = result['fixedIdx']
print("stage", stage)
print("fixedIdx", fixedIdx)
left = f'{stage}_initial'
right = f'{stage}_exception_{fixedIdx}'
print(len(json.load(open(os.path.join(dirr, f'{left}_elements.json')))), len(json.load(open(os.path.join(dirr, f'{right}_elements.json')))))
issue, (left_u, right_u) = fidelity_detect.fidelity_issue(dirr, left, right)
print(issue)
print(json.dumps(left_u, indent=2))
print(json.dumps(right_u, indent=2))

stage load
fixedIdx 0_2
1710 1753
True
[
  [
    "/html[1]/body[1]/section[1]/div[2]/header[1]/nav[1]/div[1]/a[1]",
    "/html[1]/body[1]/section[1]/div[2]/header[1]/nav[1]/div[1]/a[1]/span[1]"
  ],
  [
    "/html[1]/body[1]/section[1]/div[2]/header[1]/nav[1]/div[1]/div[1]",
    "/html[1]/body[1]/section[1]/div[2]/header[1]/nav[1]/div[1]/div[1]/ul[1]",
    "/html[1]/body[1]/section[1]/div[2]/header[1]/nav[1]/div[1]/div[1]/ul[1]/li[1]",
    "/html[1]/body[1]/section[1]/div[2]/header[1]/nav[1]/div[1]/div[1]/ul[1]/li[2]",
    "/html[1]/body[1]/section[1]/div[2]/header[1]/nav[1]/div[1]/div[1]/ul[1]/li[3]",
    "/html[1]/body[1]/section[1]/div[2]/header[1]/nav[1]/div[1]/div[1]/ul[1]/li[4]",
    "/html[1]/body[1]/section[1]/div[2]/header[1]/nav[1]/div[1]/div[1]/ul[1]/li[5]",
    "/html[1]/body[1]/section[1]/div[2]/header[1]/nav[1]/div[1]/div[1]/ul[1]/li[6]"
  ],
  [
    "/html[1]/body[1]/section[1]/div[2]/header[1]/nav[1]/div[1]/div[2]",
    "/html[1]/body[1]/section[1]/div[2]/header[1]/nav[

### Screenshot similarity

In [26]:
dirr = '../revert_rewrite/writes/test'
# dirr = '../revert_rewrite/test/load_override/writes/spanish.malabo.usembassy.gov_11513'
fixedIdx = json.load(open(os.path.join(dirr, 'result_log.json'), 'r'))["fixedIdx"]
initial = os.path.join(dirr, 'initial.png')
exception = os.path.join(dirr, f'exception_{fixedIdx}.png')

def compare_screenshot(live_img, archive_img):
    if not os.path.exists(live_img) or not os.path.exists(archive_img):
        return -1
    img1 = cv2.imread(live_img)
    img2 = cv2.imread(archive_img)
    height = min(img1.shape[0], img2.shape[0])
    width = min([img1.shape[1], img2.shape[1]])
    img1 = img1[:height,:width,:]
    img2 = img2[:height,:width,:]
    diff = img1 - img2
    total = img1.shape[0]*img1.shape[1]*img1.shape[2]
    same = np.count_nonzero(diff == 0)
    return same / total, diff

simi, diff = compare_screenshot(initial, exception)
print(simi)
cv2.imwrite(os.path.join(dirr, 'diff.png'), diff)

1.0


True

### Diff writes

In [30]:
dirr = '../revert_rewrite/writes/www.nytimes.com_1073_3_aebb2a70fcc39ea0202fcedcc6e8c81852149174e479d139da024d11887cf02b'
# dirr = '../revert_rewrite/test/load_override/writes/grunt.ca'
stage = 'load'
idx = '0_2'
attribute = 'rawWrites'
left_writes = json.load(open(os.path.join(dirr, f'{stage}_initial_writes.json')))
right_writes = json.load(open(os.path.join(dirr, f'{stage}_exception_{idx}_writes.json')))
print("Length of writes", len(left_writes[attribute]), len(right_writes[attribute]))
left_unique, right_unique = check_utils.diff_writes(left_writes, right_writes, attribute=attribute)
# len(left_unique), len(right_unique)
list(left_unique.keys()), list(right_unique.keys())

Length of writes 42 43


([('head', 'script', 'appendChild'),
  ('head', 'style', 'insertBefore'),
  ('div', 'div', 'removeChild')],
 [('head', 'link', 'appendChild'),
  ('div', 'div', 'appendChild'),
  ('n.length;s++)o=n', 'iframe', '#text', 'insertBefore')])

In [32]:
fixed = json.load(open('../revert_rewrite/fixed_count.json'))
attribute = 'rawWrites'
for dirr, stage in fixed.items():
    print(dirr, stage)
    all_dirr = f'../revert_rewrite/writes/{dirr}'
    result = json.load(open(os.path.join(all_dirr, 'results.json'), 'r'))[stage]
    idx = result['fixedIdx']
    left_writes = json.load(open(os.path.join(all_dirr, f'{stage}_initial_writes.json')))
    right_writes = json.load(open(os.path.join(all_dirr, f'{stage}_exception_{idx}_writes.json')))
    left_unique, right_unique = check_utils.diff_writes(left_writes, right_writes, attribute=attribute)
    print(len(left_unique), len(right_unique))

antimundo.org_702_2_5a7caff81b39d21034316034ac713eb594e6d8d2748d464786f7e03af80c6c72 load
0 0
smithsonianapa.org_884_0_05e534a1579f85e97ce6bee2047b6e349ac59e1860e2b733145013a4b0a13a4b load
0 0
www.paulkasmingallery.com_641_3_4c30cf2b47f220b75ebeb15b727ca9b0594ed2a2a1a778d3e700940540db2017 load
0 3
www.moma.org_531_2_4da7841d3805c7dd84abdfb099872933d3f95a24f851dbfc51492241402c0fa5 interaction_4
0 0
festival.si.edu_740_1_91bd2d5a28ffde09b540e5f3db1033287aeaced2bec240c3a159e605440af341 interaction_0
0 0
sassas.org_875_0_3041e2c8680579738802ac328f56337b7ee1c5a27c04ddfe00378be1c2f51fd4 load
0 0
www.moma.org_309_0_dbb4363b8dd9feae3b609edae0fc9d794c0e24ab71e60f1ebe4250d89533d1ba extraInteraction
13 37
www.moma.org_447_1_c4ffd66ad48ad3c5633f4a07db84e1c0eeb9500897974e2684ec2eee8462fe5d load
12 12
www.moma.org_356_1_7ec3e5a3dee8eb4f7b9c506c1497332a7950ad6783e4dfce22117cbbc1884082 load
12 12
www.moma.org_487_1_8d2b921e5f6d87ed6c1ca85222df9810a04d3c554740c93b1537cfc57b4f1443 extraInteraction
13 37

### fidelity check on a set of loads

In [14]:
def valid_dir(dirr):
    valid = os.path.exists(f'{dirr}/live_elements.json') and os.path.exists(f'{dirr}/archive_elements.json')
    return valid

base = '../record_replay/writes'
data = json.load(open(f'{base}/fidelity_check_metadata.json', 'r'))
results = []
for url, info in list(data.items()):
    dirr = info['directory']
    if not valid_dir(f"{base}/{dirr}"):
        continue
    issue, _ =  fidelity_detect.fidelity_issue(f"{base}/{dirr}")
    issue_screenshot, simi = fidelity_detect.fidelity_issue_screenshot(f"{base}/{dirr}")
    results.append({
        'directory': dirr,
        'issue': issue,
        'issue_screenshot': issue_screenshot,
        'screenshot_similarity': simi
    })
json.dump(results, open('fidelity_check.json', 'w+'), indent=2)

### fidelity check (single)

In [25]:
base = '/vault-swift/jingyz/fidelity-files/writes/ground_truth/'
dirr = 'tech.ed.gov'
issue, (left_u, right_u) = fidelity_detect.fidelity_issue(os.path.join(base, dirr), 'live', 'archive')
print(issue)
print(json.dumps(left_u, indent=2))
print(json.dumps(right_u, indent=2))

False
[]
[]
